In [1]:
import numpy as np
from scipy.linalg import logm
import pandas as pd
from sympy import *
dim = 4

sigma_z = np.array([[1, 0],[0, -1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
identity = np.identity(2)
sigmas = [identity, sigma_x, sigma_y, sigma_z]
sigmas_4d = np.kron(sigmas, sigmas)

# epsilon=0.0000001
# oepsilon=1-epsilon

def random_hamiltonian_and_rho(beta):
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    eigenval,eigenvecs=np.linalg.eig(H)
    eigenval=np.e**(-beta*eigenval)
    eigenval=np.diag(eigenval)
    s=eigenval.dot(eigenvecs.conj().T)
    rho=eigenvecs.dot(s)
    rho=rho/np.trace(rho)
    return H, rho

def pure(beta):
    H=np.random.uniform(-1,1,size=dim)
    rho=np.e**(-beta*H)
    H=np.diag(H)
    rho=np.diag(rho)
    rho/=np.trace(rho)
    return H,rho

def matrix_to_free_parameters(matr):
    free_parameters = np.real(np.array([np.trace(matr.dot( measurement)) for measurement in sigmas_4d]))
    return free_parameters


def free_parameters_to_matrix(free_parameters):
    matr = np.zeros((4,4))+0j*np.zeros((4,4))
    for i in range(dim**2):
        matr += free_parameters[i]*sigmas_4d[i]

    return matr/4

# This function is new and generates non-equilibrium states and calculates the temperature according to the paper
def nongibbs():
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    rho=a.dot(a.conj().T)
    rho/=np.trace(rho)
    t=np.trace(H)
    h=np.sqrt(np.trace(H.dot(H)-t**2/dim))
    o1=(H-t/dim)/h
    beta=np.trace(o1.dot(np.log(rho)))/(-h)
    return H,rho,beta
# a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
# H = (a.conj().T + a)/2
# print(H - free_parameters_to_matrix(matrix_to_free_parameters(H)))
def cov(A, B):
    d = 4

    out =  np.trace(np.matmul(A,B))/d - np.trace(A)*np.trace(B) / d**2

    return out

def var_matrix(A):
    d = 4
    
    out = np.trace(np.matmul(A,A))/d - (np.trace(A)/d)**2

    return out

def non_thermal():
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    rho=a.dot(a.conj().T)
    rho/=np.trace(rho)

    beta = np.real(cov(H, -logm(rho)) / var_matrix(H) )

    return H,rho,beta

hamiltonian , rho = random_hamiltonian_and_rho(0.9)
t=np.trace(hamiltonian)
h=np.sqrt(np.trace(hamiltonian.dot(hamiltonian)-t**2/dim))
o1=(hamiltonian-t/dim)/h
betta=np.trace(o1.dot(np.log(rho)))/(-h)
# p,r=
betta = np.real(cov(hamiltonian, -logm(rho)) / var_matrix(hamiltonian) )
print(betta)
# print(rho - sp.linalg.expm(-0.5*hamiltonian)/np.trace(sp.linalg.expm(-0.5*hamiltonian)))


# a = np.array([1,2,3])
# print(np.outer(a, a.conj()))



0.9000000000000008


In [2]:
numdata = 10**5




cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')

# H=np.zeros((numdata,dim,dim),dtype=np.complex_)
# rho=np.zeros((numdata,dim,dim),dtype=np.complex_)
beta = np.random.uniform(0, 1, numdata)

hamiltonian, rho = random_hamiltonian_and_rho(beta[0])

free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)

data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[0])).reshape(1, 33)


# print(len(data), data)

df = pd.DataFrame(data, columns=cols)
print(df)
for i in range(1,numdata//10):
    hamiltonian,rho=pure(beta[i])
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data
for i in range(numdata//10,numdata):
    # print(i)
    hamiltonian, rho = random_hamiltonian_and_rho(beta[i])

    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)

    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data



df.to_csv('QT - dimension 4.csv')

# for i in range(numdata):
#     H[i],rho[i]=hermitian(dim,beta[i])
# np.savez_compressed('Hamiltonians',H)
# np.savez_compressed('density matrices',rho)

   fp hamiltonian1  fp hamiltonian2  fp hamiltonian3  fp hamiltonian4  \
0         2.465892         1.343422        -0.118425         -1.00923   

   fp hamiltonian5  fp hamiltonian6  fp hamiltonian7  fp hamiltonian8  \
0         0.596291        -1.619807        -0.306634         1.881944   

   fp hamiltonian9  fp hamiltonian10  ...   fp rho8   fp rho9  fp rho10  \
0          0.53807          1.609745  ... -0.181082 -0.031161 -0.139741   

   fp rho11  fp rho12  fp rho13  fp rho14  fp rho15  fp rho16      beta  
0  0.120231 -0.018221  0.004069 -0.070174 -0.011468 -0.065247  0.375933  

[1 rows x 33 columns]


In [3]:
cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')
numdata=2*10**4
hamiltonian, rho, betas = non_thermal()
free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)
print(betas,betas.real)
data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real)).reshape(1, 33)
df = pd.DataFrame(data, columns=cols)
for i in range(1,numdata):
    hamiltonian, rho,betas = non_thermal()
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real))
    df.loc[len(df)] = data
df.to_csv('nonthermal - dimension 4.csv')

-0.45966074630068154 -0.45966074630068154
